In [1]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 4.2 MB/s 
     |████████████████████████████████| 596 kB 45.4 MB/s 
     |████████████████████████████████| 6.6 MB 38.5 MB/s 
     |████████████████████████████████| 895 kB 46.8 MB/s 
     |████████████████████████████████| 77 kB 6.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import warnings
warnings.filterwarnings(action='ignore')

In [4]:
import pandas as pd
import numpy as np

from transformers import FunnelTokenizerFast, FunnelModel, AdamW, FunnelForSequenceClassification, get_cosine_schedule_with_warmup
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from tqdm.notebook import tqdm

In [5]:
device = torch.device("cuda:0")

@misc{kim2020lmkor,
  author = {Kiyoung Kim},
  title = {Pretrained Language Models For Korean},
  year = {2020},
  publisher = {GitHub},
  howpublished = {\url{https://github.com/kiyoungkim1/LMkor}}
}

In [6]:
tokenizer = FunnelTokenizerFast.from_pretrained("kykim/funnel-kor-base")
model = FunnelForSequenceClassification.from_pretrained("kykim/funnel-kor-base")

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/336k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/153 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/725 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/709M [00:00<?, ?B/s]

Some weights of the model checkpoint at kykim/funnel-kor-base were not used when initializing FunnelForSequenceClassification: ['decoder.layers.1.attention.q_head.weight', 'decoder.layers.1.ffn.layer_norm.weight', 'decoder.layers.1.attention.v_head.bias', 'decoder.layers.0.attention.post_proj.bias', 'decoder.layers.1.attention.seg_embed', 'decoder.layers.1.ffn.linear_1.bias', 'decoder.layers.0.ffn.linear_1.weight', 'decoder.layers.1.attention.k_head.bias', 'decoder.layers.0.attention.q_head.weight', 'decoder.layers.1.attention.layer_norm.weight', 'decoder.layers.0.attention.layer_norm.bias', 'decoder.layers.0.attention.k_head.bias', 'decoder.layers.1.attention.r_kernel', 'decoder.layers.1.attention.r_s_bias', 'decoder.layers.0.attention.v_head.bias', 'decoder.layers.0.ffn.layer_norm.bias', 'decoder.layers.0.attention.r_r_bias', 'decoder.layers.0.attention.r_s_bias', 'decoder.layers.1.ffn.linear_2.bias', 'decoder.layers.1.attention.post_proj.bias', 'decoder.layers.0.attention.layer_norm

In [7]:
model.classifier.linear_out = nn.Sequential(nn.Linear(768, 1),
                                            nn.Sigmoid())

In [8]:
model

FunnelForSequenceClassification(
  (funnel): FunnelBaseModel(
    (embeddings): FunnelEmbeddings(
      (word_embeddings): Embedding(42000, 768)
      (layer_norm): LayerNorm((768,), eps=1e-09, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): FunnelEncoder(
      (attention_structure): FunnelAttentionStructure(
        (sin_dropout): Dropout(p=0.1, inplace=False)
        (cos_dropout): Dropout(p=0.1, inplace=False)
      )
      (blocks): ModuleList(
        (0): ModuleList(
          (0): FunnelLayer(
            (attention): FunnelRelMultiheadAttention(
              (hidden_dropout): Dropout(p=0.1, inplace=False)
              (attention_dropout): Dropout(p=0.1, inplace=False)
              (q_head): Linear(in_features=768, out_features=768, bias=False)
              (k_head): Linear(in_features=768, out_features=768, bias=True)
              (v_head): Linear(in_features=768, out_features=768, bias=True)
              (post_proj): Linear(i

In [9]:
PATH = '/content/gdrive/MyDrive/Colab Notebooks/project/'
cate = 'clean'
df = pd.read_csv(PATH + f'{cate}.csv')

In [10]:
df = df[['문장', f'{cate}']]

In [11]:
from torch.utils.data import Dataset, TensorDataset, DataLoader, random_split
from torch import torch
from sklearn.model_selection import train_test_split

def dataSplit(dataset, y_label):
  X_train, X_val= train_test_split(dataset, test_size = 0.2, stratify = dataset[y_label], random_state =427)
  return X_train, X_val

In [12]:
X_train, X_test = dataSplit(df, cate)
X_train, X_val = dataSplit(X_train, cate)

In [13]:
class LoadDataset(Dataset):
    def __init__(self, df, tk):
        self.df = df
        self.tokenizer = tk

    def __len__(self):
        return len(self.df)
  
    def __getitem__(self, idx):
        row = self.df.iloc[idx, :].values
        # target이 없는경우 (즉, 문장만 입력된 경우)
        if len(row) <= 1:
            text = row[0]

            inputs = self.tokenizer(
                text, 
                return_tensors='pt',
                truncation=True,
                max_length=50,
                pad_to_max_length=True,
                add_special_tokens=True
                )
            
            input_ids = inputs['input_ids'][0].to(device)
            attention_mask = inputs['attention_mask'][0].to(device)

            return input_ids, attention_mask     
            
        # target이 있는 경우 (원래 코드)
        else:
            text = row[0]
            y = row[1]

            inputs = self.tokenizer(
                text, 
                return_tensors='pt',
                truncation=True,
                max_length=50,
                pad_to_max_length=True,
                add_special_tokens=True
                )
            
            input_ids = inputs['input_ids'][0].to(device)
            attention_mask = inputs['attention_mask'][0].to(device)

            return input_ids, attention_mask, y

In [14]:
train_set = LoadDataset(X_train, tokenizer)
val_set = LoadDataset(X_val, tokenizer)
test_set = LoadDataset(X_test, tokenizer)

In [15]:
epochs = 300 # epochs 증가
batch_size = 32
warmup_ratio=0.1
t_total = len(train_set) * epochs
optimizer = AdamW(model.parameters(), lr=1e-5, eps = 1e-8) # lr 1/10으로 변경
train_loader = DataLoader(train_set, batch_size=batch_size)
val_loader = DataLoader(val_set, batch_size=batch_size) # val loader 추가
test_loader = DataLoader(test_set, batch_size = batch_size) # test loader 추가
loss_f = nn.BCEWithLogitsLoss() # loss f 변경

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=1, num_training_steps=t_total)

In [16]:
# https://github.com/Bjarten/early-stopping-pytorch
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, path=f'/content/gdrive/MyDrive/checkpoint_{cate}.pt'):
        """
        Args:
            patience (int): validation loss가 개선된 후 기다리는 기간
                            Default: 7
            verbose (bool): True일 경우 각 validation loss의 개선 사항 메세지 출력
                            Default: False
            delta (float): 개선되었다고 인정되는 monitered quantity의 최소 변화
                            Default: 0
            path (str): checkpoint저장 경로
                            Default: 'checkpoint.pt'
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [17]:
early_stopping = EarlyStopping(patience = 7, verbose = True)

In [18]:
model = model.cuda()

In [19]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [20]:
from sklearn.metrics import roc_auc_score

for i in tqdm(range(epochs)):
    train_loss_list = [] # 변수 변경
    val_loss_list = []
    val_score_list = []

    epoch_train_loss = []
    epoch_val_loss = []
    epoch_val_score = []
    # train
    model.train()
    for input_ids_batch, attention_masks_batch, y_batch in train_loader:
        input_ids_batch = input_ids_batch
        attention_masks_batch = attention_masks_batch
        y_batch = y_batch
        optimizer.zero_grad()
        y_pred = model(input_ids_batch, attention_mask=attention_masks_batch).logits.reshape(-1)
#        print(y_pred)
        loss = loss_f(y_pred.type(torch.FloatTensor), y_batch.type(torch.FloatTensor))
#        print(loss)
        loss.backward()
        optimizer.step()
        scheduler.step()
        train_loss_list.append(loss.item())

    # validation loss
    model.eval()
    for input_ids_batch_val, attention_masks_batch_val, y_batch_val in val_loader:
        input_ids_batch_val = input_ids_batch_val
        attention_masks_batch_val = attention_masks_batch_val
        y_batch_val = y_batch_val
        y_pred_val = model(input_ids_batch_val, attention_mask = attention_masks_batch_val).logits.reshape(-1)
        loss = loss_f(y_pred_val.type(torch.FloatTensor), y_batch_val.type(torch.FloatTensor))
        val_score = roc_auc_score(y_batch_val.tolist(), y_pred_val.tolist())
        val_loss_list.append(loss.item())
        val_score_list.append(val_score)

    # epoch당 loss 계산 (for early stopping)
    train_loss = np.average(train_loss_list)
    val_loss = np.average(val_loss_list)
    val_score = np.average(val_score_list)

    epoch_train_loss.append(train_loss)
    epoch_val_loss.append(val_loss)
    epoch_val_score.append(val_score)
    epoch_len = len(str(epochs))

    print_msg = (f'[{i:>{epoch_len}}/{epochs:>{epoch_len}}] ' +
                 f'train_loss: {train_loss:.5f} ' +
                 f'valid_loss: {val_loss:.5f} ' +
                 f'valid_score: {val_score:.5f}')

    print(print_msg)
    
    # clear lists to track next epoch
    train_loss_list = []
    val_loss_list = []
    val_score_list = []
    early_stopping(val_loss, model)
    if early_stopping.early_stop:
        print('early stopping')
        break
    
model.load_state_dict(torch.load(f'/content/gdrive/MyDrive/checkpoint_{cate}.pt', map_location=device))

  0%|          | 0/300 [00:00<?, ?it/s]

[  0/300] train_loss: 0.64536 valid_loss: 0.60024 valid_score: 0.85875
Validation loss decreased (inf --> 0.600243).  Saving model ...
[  1/300] train_loss: 0.59318 valid_loss: 0.59656 valid_score: 0.86108
Validation loss decreased (0.600243 --> 0.596563).  Saving model ...
[  2/300] train_loss: 0.58277 valid_loss: 0.58730 valid_score: 0.88760
Validation loss decreased (0.596563 --> 0.587297).  Saving model ...
[  3/300] train_loss: 0.57360 valid_loss: 0.59839 valid_score: 0.87237
EarlyStopping counter: 1 out of 7
[  4/300] train_loss: 0.57684 valid_loss: 0.58978 valid_score: 0.87629
EarlyStopping counter: 2 out of 7
[  5/300] train_loss: 0.57178 valid_loss: 0.59671 valid_score: 0.86893
EarlyStopping counter: 3 out of 7
[  6/300] train_loss: 0.56893 valid_loss: 0.59001 valid_score: 0.86271
EarlyStopping counter: 4 out of 7
[  7/300] train_loss: 0.56200 valid_loss: 0.58724 valid_score: 0.88760
Validation loss decreased (0.587297 --> 0.587242).  Saving model ...
[  8/300] train_loss: 0.5

<All keys matched successfully>

In [21]:
model.eval()
valid_loader = DataLoader(val_set, batch_size=batch_size)
score_list = []
for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
    input_ids_batch = input_ids_batch
    attention_masks_batch = attention_masks_batch
    y_batch = y_batch
    y_pred = model(input_ids_batch, attention_mask=attention_masks_batch).logits.reshape(-1)
#    print(y_pred)
    try:
        score = roc_auc_score(y_batch.tolist(), y_pred.tolist())
        score_list.append(score)
    except: pass


print("epoch roc_auc:", np.mean(score_list))

  0%|          | 0/59 [00:00<?, ?it/s]

epoch roc_auc: 0.8817237643582622


In [22]:
# score 기록하기
final_score = np.mean(score_list)

In [23]:
#korean_col_name_list = ['clean','local','religion_model','']
#eng_model_name_list = ['clean','지역','종교','인종국적','연령','악플욕설','성소수자','성별','기타혐오','개인지칭']

#translate_dict = {}

torch.save(model.state_dict(), PATH + f'{cate}.pth')

In [24]:
print(final_score, cate)

0.8817237643582622 clean
